In [1]:
import ee
import geemap
import time
import requests
import json
import os
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd
import topojson as tp
#ee.Authenticate()
#geemap.ee_search()
ee.Initialize()
Map = geemap.Map()

In [2]:
countries = ee.FeatureCollection('users/raulperezastorga/Limites_SPPP')
roi = countries

In [3]:
t2=2024
t1=t2-1
t2=str(t2)
t1=str(t1)

In [4]:
prefire_start = f'{t1}-07-01'
prefire_end = f'{t2}-02-28'

# Now set the same parameters for AFTER the fire.
postfire_start = f'{t2}-03-01'
#postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
postfire_end = f'{t2}-06-30'

In [5]:
prefire_start

'2023-07-01'

In [6]:
vizfalsoColor = {
  'bands': ['B8', 'B4B', 'B3'],
  'min': 0 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

vizfalsoColor2 = {
  'bands': ['B8', 'B4', 'B3'],
  'min': 0 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

vizfire = {
  'bands': ['B11', 'B8B', 'B4'],
  'min': 0.17 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

In [7]:
##functiones prepare data reduce positive false and false positive
#dNBR for s2, we recomend use s2_sr_harmonized for better, the normalizeddifference between nir and swir
# s2 nir=B8, swir = B12
def addNBR_s2h(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image
# l8 nir=B5, swir=B7
def addNBR_l8(image):
    nbr = image.normalizedDifference(['B5', 'B7']).rename("nbr")
    image = image.addBands(nbr)
    return image
# l5 and l7 nir=B4, swir B7
def addNBR_l5(image):
    nbr = image.normalizedDifference(['B4', 'B7']).rename("nbr")
    image = image.addBands(nbr)
    return image


## define a function that masks clouds
# for s2
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked image, scaled to [0, 1].
  return image.updateMask(mask).divide(10000)

# in landsat 8 
# Define a function to apply the cloud mask
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
  
    # Get the pixel QA band
    qa = image.select('pixel_qa')
  
    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) and (qa.bitwiseAnd(cloudsBitMask)).eq(0)

    # Return the masked image with updated metadata
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# in Landsat 5 images
def maskL5sr(image):
    # Bits 3, 4, and 5 are cloud, cloud shadow, and snow/ice, respectively.
    cloudsBitMask = (1 << 3)
    cloudShadowBitMask = (1 << 4)
    snowIceBitMask = (1 << 5)
  
    # Get the pixel QA band
    qa = image.select('pixel_qa')
  
    # All flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloudsBitMask).eq(0) and \
           qa.bitwiseAnd(cloudShadowBitMask).eq(0) and \
           qa.bitwiseAnd(snowIceBitMask).eq(0)

    # Return the masked image with updated metadata
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

###fire masks
#better option is modis MCD64A1 but is avaible 3 or 4 months before wildfire


def mask_fires_modis_mcd64a1(image):
    collection_firms = ee.ImageCollection("MODIS/061/MCD64A1") \
        .filterBounds(roi) \
        .filterDate(prefire_start, postfire_end) \
        .select('BurnDate')
    fire_mask = collection_firms.max().gte(0).rename('fires')
    return image.updateMask(fire_mask)

# Filter fires by FIRMS dataset
def mask_fire_firms(image):
    firms = ee.ImageCollection('FIRMS')
    fires_firms = ee.ImageCollection(firms\
            .select('T21')\
            .filterDate(prefire_start,postfire_end)\
            .filterBounds(roi)) 
    fires_firmsmx = fires_firms.max().clip(roi).subtract(273.15)
    fires_firmsrc = fires_firmsmx.gte(100)
    return image.updateMask(fires_firmsrc)

## Vegetation mask before fire
# Load MODIS Land Cover Type Yearly Global dataset (MCD12Q1) for year 2019

# Get the vegetation/bare cover band


# Define a function to apply the vegetation mask to an input image
def maskVegetation(image):
    mcd12q1 = ee.ImageCollection('MODIS/006/MCD12Q1') \
        .filter(ee.Filter.date(prefire_start, prefire_end)) \
        .filterBounds(roi)\
        .first()
    vg = mcd12q1.select('LC_Type1')
    # Get the pixels that represent vegetation
    mask = vg.eq(1).Or(vg.eq(2)).Or(vg.eq(3)).Or(vg.eq(4)).Or(vg.eq(5)).Or(vg.eq(6)).Or(vg.eq(7))
    
    # Return the masked image with updated metadata
    return image.updateMask(mask)

# mask elevation
def mask_elevation(image):
    elevation_data = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
    elevation_mask = elevation_data.lt(3500)
    return image.updateMask(elevation_mask)

In [8]:
if int(t2)>2018:
    ImCol='COPERNICUS/S2_SR_HARMONIZED'
else:
    ImCol='COPERNICUS/S2'
print(ImCol)
#from 2016
prefireImCol = ee.ImageCollection(ImCol) \
                  .filterDate(prefire_start, prefire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .sort('CLOUDY_PIXEL_PERCENTAGE')\
                  .filterBounds(roi) \
                  .sort('system:time_start')\
                  .map(maskS2clouds)\
                  .map(mask_elevation)\
                  .map(mask_fires_modis_mcd64a1)\
                  .map(addNBR_s2h)
                  
postfireImCol = ee.ImageCollection(ImCol) \
                  .filterDate(postfire_start, postfire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
                  .filterBounds(roi) \
                  .map(maskS2clouds)\
                  .map(mask_elevation)\
                  .map(mask_fires_modis_mcd64a1)\
                  .map(addNBR_s2h)\
                  .sort('system:time_start', False)


#HARMONIZE
prefire = prefireImCol.max().clip(roi)
postfire = postfireImCol.mosaic().clip(roi)

COPERNICUS/S2_SR_HARMONIZED


In [9]:
preNBR = prefire.select('nbr')
postNBR = postfire.select('nbr')
dNBR_withoutsmooth = preNBR.subtract(postNBR)
dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=ee.Kernel.gaussian(radius=5),
)
#dNBR_unscaled=dNBR_unscaled.clip(r20)
dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
#dNBR_Seguro = dNBR_unscaled.gte(0.27)

viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}

In [10]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
Map.addLayer(dNBR_reclass, viz_dNBR, 'dNBR Clase')

In [12]:
Map.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),
 TileLayer(attribution='Google', max_zoom=22, name='Google Maps', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'),
 TileLayer(attribution='Google Earth Engine', max_zoom=24, name='dNBR Clase', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/4b

In [13]:
dNBR_vector= dNBR_reclass.reduceToVectors(**{
'geometry':roi,
'crs': 'EPSG:32718',
'scale':30,
'geometryType':'polygon',
'eightConnected': False,
'bestEffort':False,
'maxPixels':9e10,
'labelProperty': 'id_dano',
}
)
dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
#Dissolve the simplified polygons and reduce file size

    #.filterMetadata('label', 'greater_than', 3)
geemap.ee_export_vector_to_drive(
    dNDBR_export, description="dNBR_mcd64a1_pre_"+t2, fileFormat='GeoJSON', folder="dNBR"
)

Exporting dNBR_mcd64a1_pre_2023... Please check the Task Manager from the JavaScript Code Editor.


In [14]:
dNBR_vector= dNBR_reclass.reduceToVectors(**{
'geometry':roi,
'crs': 'EPSG:32718',
'scale':30,
'geometryType':'polygon',
'eightConnected': False,
'bestEffort':False,
'maxPixels':9e10,
}
)
dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))#\
    #.filterMetadata('label', 'greater_than', 3)
geemap.ee_export_vector_to_cloud_storage(
    dNDBR_export, description="dNBR_mcd64a1_mxmn_"+t2, fileFormat='csv',bucket='ee_geomatica'
)

Exporting dNBR_mcd64a1_mxmn_2023... Please check the Task Manager from the JavaScript Code Editor.
